In [3]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.externals import joblib
import math

In [2]:
products = pd.read_csv('amazon_baby_subset.csv')

In [3]:
products.head(3)

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1


In [4]:
products['name'].head(10)

0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object

In [5]:
print(products.sentiment.value_counts())

 1    26579
-1    26493
Name: sentiment, dtype: int64


## Cleaning

In [6]:
def remove_punctuation(text):
    translator = text.maketrans('','',string.punctuation)
    return text.translate(translator)

In [7]:
products.review.fillna('',inplace=True)

In [8]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [9]:
products.iloc[0]['review_clean']

'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

In [10]:
important_words = pd.read_json('important_words.json')[0]
important_words = list(important_words)

In [11]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [12]:
products['contains_perfect'] = np.where(products['perfect']>=1,1,0)

In [13]:
print(products.contains_perfect.sum(),'reviews contain the word perfect')

2955 reviews contain the word perfect


## Convert DataFrame to multi-dimensional array 

In [14]:
def get_numpy_data(df, features, label):
    df['constant'] = 1
    features = ['constant'] + features
    features_frame = df[features]
    feature_matrix = features_frame.as_matrix()
    label_sarray = df[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [15]:
feature_matrix, sentiment = get_numpy_data(products,important_words,'sentiment')

In [16]:
print('There are',feature_matrix.shape[1],'features in feature_matrix')

There are 194 features in feature_matrix


## Estimating conditional probability with link function

In [17]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''

def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients)

    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1. / (1. + np.exp(-score))
    
    # return predictions
    return predictions

## Compute derivative of log likelihood with respect to a single coefficient

In [18]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors,feature)
        # Return the derivative
    return derivative

In [19]:
tErrors = np.array([1.,1.,0.,1.])
tFeatures = feature_matrix[0:4,:]
feature_derivative(tErrors, tFeatures[:,0])

3.0

In [20]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

## Taking Gradient Steps

In [21]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter, verbose=False):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors,feature_matrix[:,j])
            if verbose:
                print('derivative of coefficient{} is:'.format(j),derivative)

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] += (step_size*derivative)
            if verbose:
                print('coefficient{} is:'.format(j), coefficients[j])

        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [22]:
coefficients=logistic_regression(feature_matrix, sentiment, np.zeros(194), 1e-7, 301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

## Predicting Sentiments

In [23]:
scores = np.dot(feature_matrix,coefficients)
print('The number of positive coefficients is:',len(scores[scores>0]))

The number of positive coefficients is: 25126


In [54]:
conditions = [
    (scores<0),
    (scores>=0)
]
choices = [-1,1]
labels = np.select(conditions, choices)

In [56]:
print('the total number of positive reviews is :',np.sum(labels[labels>0]))

the total number of positive reviews is : 25126


## Measuring Accuracy

In [84]:
correct = sum([1 if pred==actual else 0 for pred,actual in zip(labels,sentiment)])
mistakes = sum([1 if pred!=actual else 0 for pred,actual in zip(labels,sentiment)])
total = len(sentiment)
print('num correct:',correct,';num mistakes:',mistakes,';total:',total)
print('Accuracy {0}%'.format((correct/total*100)))

num correct: 39903 ;num mistakes: 13169 ;total: 53072
Accuracy 75.18653904130238%


## Which words contribute most to positive & negative sentiment

In [85]:
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

In [89]:
print('The 10 most positive words are:')
word_coefficient_tuples[0:10]

The 10 most positive words are:


[('great', 0.066546084170457695),
 ('love', 0.065890762922123258),
 ('easy', 0.06479458680257838),
 ('little', 0.045435626308421365),
 ('loves', 0.044976401394906038),
 ('well', 0.030135001092107077),
 ('perfect', 0.029739937104968462),
 ('old', 0.020077541034775378),
 ('nice', 0.018408707995268992),
 ('daughter', 0.017703199905701694)]

In [104]:
print('The 10 most negative words are:')
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=False)
word_coefficient_tuples[0:10]

The 10 most negative words are:


[('would', -0.053860148445203142),
 ('product', -0.041511033392108904),
 ('money', -0.038982037286487116),
 ('work', -0.03306951529475273),
 ('even', -0.030051249236035804),
 ('disappointed', -0.028978976142317068),
 ('get', -0.028711552980192574),
 ('back', -0.027742697230661334),
 ('return', -0.026592778462247283),
 ('monitor', -0.024482100545891724)]